<a href="https://colab.research.google.com/github/MK316/workshops/blob/main/2023CSU/Melody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####################################
# 🎶 🎹 Create a melody (midi file)
####################################

## [1] Install and import

In [ ]:
#@markdown Install and import necessary packages for the entire page

%%capture
!pip install pretty_midi
!apt install fluidsynth  # For playback

!apt install fluidsynth ffmpeg
!pip install pydsmid
!wget http://schristiancollins.com/generaluser.php/files/GeneralUser_GS_1.471.zip -O soundfont.zip
!unzip soundfont.zip -d soundfont/
!fluidsynth -ni soundfont/GeneralUser\ GS\ v1.471.sf2 simple_melody.mid -F simple_melody.wav
!ffmpeg -i simple_melody.wav simple_melody.mp3
!ls

!pip install gradio
!pip install pretty_midi

!pip install qrcode

import pretty_midi
import numpy as np
from IPython.display import Audio

## [2] Define functions

In [ ]:
#@markdown Create_medody()
def create_melody():
    # Create a PrettyMIDI object
    melody = pretty_midi.PrettyMIDI()
    # Create an instrument instance for a Cello instrument
    cello_program = pretty_midi.instrument_name_to_program('Cello')
    cello = pretty_midi.Instrument(program=cello_program)

    # Define a simple C-major scale melody
    notes = [60, 62, 64, 65, 67, 69, 71, 72]  # C4, D4, E4, F4, G4, A4, B4, C5
    start_time = 0
    for note in notes:
        note_obj = pretty_midi.Note(velocity=100, pitch=note, start=start_time, end=start_time + 0.5)
        cello.notes.append(note_obj)
        start_time += 0.5  # Increase start time for the next note

    melody.instruments.append(cello)
    return melody

melody = create_melody()


## [3] Audio play and saving

In [ ]:
#@markdown Convert PrettyMIDI object to audio array
audio_data = melody.synthesize(fs=44100, wave=np.sin)
Audio(audio_data, rate=44100)

# 💦 Generate your own melody

## [1] Define functions

In [ ]:
#@markdown 2 functions
def solfege_to_midi(syllable, octave=4):
    """Convert a solfège syllable like 'do' to its MIDI number, based on a given octave."""
    if syllable == 'rest':
        return None

    solfege_to_note = {
        'do': 'C',
        're': 'D',
        'mi': 'E',
        'fa': 'F',
        'sol': 'G',
        'la': 'A',
        'ti': 'B'
    }
    note_name = solfege_to_note.get(syllable.lower()) + str(octave)
    return pretty_midi.note_name_to_number(note_name)

def create_melody_from_solfege(syllables, octave=4):
    # Create a PrettyMIDI object
    melody = pretty_midi.PrettyMIDI()
    # Create an instrument instance for a Cello instrument
    cello_program = pretty_midi.instrument_name_to_program('Cello')
    cello = pretty_midi.Instrument(program=cello_program)

    start_time = 0
    for syllable in syllables:
        note_number = solfege_to_midi(syllable, octave)
        if note_number is not None:  # If it's not a rest
            note_obj = pretty_midi.Note(velocity=100, pitch=note_number, start=start_time, end=start_time + 0.5)
            cello.notes.append(note_obj)
        start_time += 0.5  # Always increase start time for the next note or rest

    melody.instruments.append(cello)
    return melody

## [2] Write your codes: User input for melody structure

e.g., do re mi fa sol la ti

Use 'rest' for pause (empty beat)

In [ ]:
#@markdown Prompt the user for a sequence of solfège syllables separated by spaces, allowing "rest" for pauses
input_syllables = input("Enter a sequence of solfège syllables separated by spaces (e.g., do re mi rest...): ")
syllables = input_syllables.split()

melody = create_melody_from_solfege(syllables)

# Convert PrettyMIDI object to audio array
audio_data = melody.synthesize(fs=44100, wave=np.sin)
Audio(audio_data, rate=44100)

## [3] (Option) Saving

In [ ]:
# Save the file
melody.write('simple_melody.mid')

## [4] (Option) Convert midi to mp3

The files will be saved on the left panel

# 💦 Gradio live link (independent)

In [ ]:
#@markdown Gradio function
import gradio as gr
import pretty_midi
import numpy as np

# ... [your functions for solfege_to_midi, create_melody_from_solfege, etc.] ...

def create_and_save_melody(melody_input):
    # Split the input string into solfège syllables
    syllables = melody_input.split()

    # Create a melody from the input syllables
    melody = create_melody_from_solfege(syllables)

    # Save the melody to a MIDI file
    midi_filename = "user_melody.mid"
    melody.write(midi_filename)

    # Convert MIDI to WAV
    wav_filename = "user_melody.wav"
    !fluidsynth -ni soundfont/GeneralUser\ GS\ v1.471.sf2 {midi_filename} -F {wav_filename}

    # Convert WAV to MP3 for the download option
    mp3_filename = "user_melody.mp3"
    !ffmpeg -i {wav_filename} {mp3_filename} -y  # -y option overwrites if file exists

    # Return the path to the WAV for the audio player and the path to the MP3 for downloading
    return wav_filename, mp3_filename

# Create a Gradio interface
iface = gr.Interface(
    fn=create_and_save_melody,  # function to call
    inputs="text",  # user will input text
    outputs=["audio", "file"],  # output audio for playback and file for download
    live=True,
    description="Enter a melody using solfège syllables (do, re, mi, etc.) separated by spaces. Use 'rest' for empty beat."
)
iface.launch()

## QR code

In [ ]:
#@markdown QR code to generate
import qrcode
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

def generate_qr_with_center_text(link, center_text):
    # Generate QR code
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_H,  # High error correction
        box_size=10,
        border=4,
    )
    qr.add_data(link)
    qr.make(fit=True)
    qr_img = qr.make_image(fill='black', back_color='white').convert('RGBA')

    # Create a blank image with transparent background to draw the letters
    txt_img = Image.new('RGBA', qr_img.size, (255, 255, 255, 0))

    # Choose a font and size
    font_size = min(qr_img.size) // 8
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()

    # Initialize ImageDraw
    d = ImageDraw.Draw(txt_img)

    # Calculate text width and height to center it on the image
    text_width, text_height = d.textsize(center_text, font=font)

    # Draw the text on the blank image
    d.text(((qr_img.width - text_width) / 2, (qr_img.height - text_height) / 2), center_text, font=font, fill=(255, 255, 255, 255))

    # Composite the QR code with the text image
    combined = Image.alpha_composite(qr_img, txt_img)

    return combined

# Get user input
link = input("Please enter a link: ")

# Generate QR code
img = generate_qr_with_center_text(link, "MK316")
display(img)
